## Importing Necessary Libraries

In [1]:
# Python available modulses
import glob
import os
import textract
from gensim.summarization.summarizer import summarize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from os.path import isfile, join
from io import StringIO
from collections import Counter
import nltk
nltk.download('wordnet')

import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import PhraseMatcher

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# Developed Module
import text_process

/Users/nithintata/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nithintata/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Global Variables to store resume paths

In [3]:
resume_list = [] # stores all resumes
resume_list_pdf = [] # Captures files with pdf extension
resume_list_doc = [] # Captures files with doc extension
resume_list_docx = [] # Captures files with docx extension
 
file_names = [] # STORES RESUME FILE NAMES
job_desc_files = [] # stores jd paths

#path = 'C:/Users/sampathi/PycharmProjects/Resume_Ranking/Automated-Resume-Ranking-System-master/Original_Resumes'
path= '/Users/nithintata/Documents/GitHub/Capestone-project-group3/Original_Resumes'
for file in glob.glob(path + '/*.pdf', recursive=True):
    resume_list_pdf.append(file)
for file in glob.glob('**/*.doc', recursive=True):
    resume_list_doc.append(file)
for file in glob.glob('**/*.docx', recursive=True):
    resume_list_docx.append(file)

resume_list = resume_list_doc + resume_list_docx + resume_list_pdf

In [5]:
import requests
resume_list1=[]
a=requests.get("https://firestore.googleapis.com/v1/projects/capestone-945f7/databases/(default)/documents/jobsApplied").json()
for aa in a["documents"]:
    resume_list1.append(aa["fields"]["url"]["stringValue"])

In [6]:
resume_list1

['https://firebasestorage.googleapis.com/v0/b/capestone-945f7.appspot.com/o/eyBbM03W6x2ckvJemmjg%2FNithin%20Tatafront-end%20dev%20resume%20sample.pdf?alt=media&token=e5221fdc-4f57-45fc-b0c2-c6efe2b03e69',
 'https://firebasestorage.googleapis.com/v0/b/capestone-945f7.appspot.com/o/0q1yyOoSTOQKSJt1yDeh%2FJaswanth%20TataFinal_CV.pdf?alt=media&token=9b192e75-9c27-4574-bf11-6d7f7ccc743d',
 'https://firebasestorage.googleapis.com/v0/b/capestone-945f7.appspot.com/o/0q1yyOoSTOQKSJt1yDeh%2FNithin%20TataJaswanth.pdf?alt=media&token=dc2803f6-8101-4f4a-8869-fe6604ecf405',
 'https://firebasestorage.googleapis.com/v0/b/capestone-945f7.appspot.com/o/z2sLDgTFqKtznZ49aFWM%2FNithin%20Tatasql%20dev%20resume%20sample.pdf?alt=media&token=4cc6b526-ece7-4981-b700-f4e086486e0b',
 'https://firebasestorage.googleapis.com/v0/b/capestone-945f7.appspot.com/o/eyBbM03W6x2ckvJemmjg%2FNithin%20TataAlice%20Clark%20CV.pdf?alt=media&token=e7cf03b9-d784-4e4c-8496-925dd5953bf9',
 'https://firebasestorage.googleapis.com/v0/

## Extract and Preprocess the text in Resume

In [7]:
def extract_text_from_pdf(files_list):
    resumes = [] # Stores final processed resume files 
    for pdf_path in files_list:
        text = ''
        with open(pdf_path, 'rb') as fh:
            # iterate over all pages of PDF document
            for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
                # creating a resoure manager
                resource_manager = PDFResourceManager()

                # create a file handle
                fake_file_handle = StringIO()

                # creating a text converter object
                converter = TextConverter(
                                    resource_manager, 
                                    fake_file_handle, 
                                    codec='utf-8', 
                                    laparams=LAParams()
                            )

                # creating a page interpreter
                page_interpreter = PDFPageInterpreter(
                                    resource_manager, 
                                    converter
                                )

                # process current page
                page_interpreter.process_page(page)

                # extract text
                text += fake_file_handle.getvalue()
                text = text.replace('\n', ' ')
                
                # close open handles
                converter.close()
                fake_file_handle.close()
            resumes.append(text_process.normalize(text))
            
    for name in resume_list:
        temp = name.split('.')[0]
        temp = temp.split('/')[-1]
        file_names.append(temp)
    df = {'Path':resume_list, 'File Name': file_names, 'Text':resumes}
    data = pd.DataFrame(df)
    return data

## Parsing the Job Description(jd)

In [8]:
#file_loc = 'C:/Users/sampathi/PycharmProjects/Resume_Ranking/Automated-Resume-Ranking-System-master/Job_Description/'
file_loc = '/Users/nithintata/Documents/GitHub/Capestone-project-group3/Original_Resumes/'
def parsing_jd(jd_file_name):
    path = file_loc + jd_file_name + '.txt'
    for file in glob.glob(path, recursive=True):
        if not file in job_desc_files: 
            job_desc_files.append(file)
    with open(path, 'rt') as file:
        jd = file.read()
    jd = summarize(jd, word_count=200)
    file.close()
    jd = text_process.normalize(jd)
    
#     dict = {'Path':path, 'File Name': jd_file_name, 'Text':jd}
#     df = pd.DataFrame(dict)
    
    df = pd.DataFrame(columns=['Path', 'File Name', 'Text'])
    df.loc[0] = [path, jd_file_name, jd]
    return df

## Return the resumes close to JD

In [9]:
def resume_df(files_list, jd_file_name):
    
    df1 = extract_text_from_pdf(files_list)
    df2 = parsing_jd(jd_file_name)
    df3 = pd.concat([df1, df2], ignore_index = True)
    
    tfidfVect = TfidfVectorizer()
    tfidf = tfidfVect.fit_transform(df3['Text'])
    job_desc = df3[df3['File Name'] == jd_file_name]
    
    jd_tfidfVect = TfidfVectorizer()
    jd_tfidfVect = jd_tfidfVect.fit(df3['Text'])
    jd_tfidf = jd_tfidfVect.transform(job_desc['Text'])
    
#     feature_array = np.array(feature_names)
#     tfidf_sorting = np.argsort(jd_tfidf.toarray()).flatten()[::-1]
#     top_n = feature_array[tfidf_sorting][:10]
    
    nbrs = NearestNeighbors(n_neighbors=5).fit(tfidf)
    distances, indices = nbrs.kneighbors(jd_tfidf)
    names_similar = pd.Series(indices.flatten()).map(df3.reset_index()['File Name'])
    result = pd.DataFrame({'Distance':distances.flatten(), 'Resume':names_similar})
    
    return result[1:]

In [31]:
#resume_list
import tempfile
for file_url in zip(resume_list1,range(0,len(resume_list1))):
    response=requests.get(file_url[0])
    #fine_name=os.path.join(tempfile.gettempdir(),"hello"+str(file_url[1]))
    fine_name="hello"+str(file_url[1])+".pdf"
    print(file_url[0].split("/")[-1].split("?")[0])
    with open(fine_name, 'wb') as f:
        f.write(response.content)
    print("Cpmpleted")

eyBbM03W6x2ckvJemmjg%2FNithin%20Tatafront-end%20dev%20resume%20sample.pdf
Cpmpleted
0q1yyOoSTOQKSJt1yDeh%2FJaswanth%20TataFinal_CV.pdf
Cpmpleted
0q1yyOoSTOQKSJt1yDeh%2FNithin%20TataJaswanth.pdf
Cpmpleted
z2sLDgTFqKtznZ49aFWM%2FNithin%20Tatasql%20dev%20resume%20sample.pdf
Cpmpleted
eyBbM03W6x2ckvJemmjg%2FNithin%20TataAlice%20Clark%20CV.pdf
Cpmpleted
z2sLDgTFqKtznZ49aFWM%2FNithin%20TataAlice%20Clark%20CV.pdf
Cpmpleted
0q1yyOoSTOQKSJt1yDeh%2Fjaswanth%20tataResume%20(6).pdf
Cpmpleted
eyBbM03W6x2ckvJemmjg%2FJaswanth%20TataJalpaDave_CV.pdf
Cpmpleted


In [22]:
resume_list1[0].split("/")[-1].split("?")[0]

'eyBbM03W6x2ckvJemmjg%2FNithin%20Tatafront-end%20dev%20resume%20sample.pdf'

In [32]:
resume_list2=["hello1.pdf","hello2.pdf","hello3.pdf","hello4.pdf","hello5.pdf","hello6.pdf","hello7.pdf"]
resume_df(resume_list2, 'sample')

ValueError: arrays must all be same length

### Cosine Similarity
### Skill API 
### Score with Phrase Matching
### Suggested Job Titles
### Distance, Cosine Similarity, Lavensh Distance, Resume, Jod desc ID, Resume ID, Overall score